In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## read in the data
animes = pd.read_csv('anime.csv')
ratings = pd.read_csv('rating.csv')

# Data Cleaning

In [2]:
print(animes.shape)
animes.head(10)

(12294, 7)


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109


In [3]:
print(ratings.shape)
ratings.head(10)

(7813737, 3)


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
5,1,355,-1
6,1,356,-1
7,1,442,-1
8,1,487,-1
9,1,846,-1


In [4]:
## only keep TV animes for this recommender system
animes = animes[animes['type'] == 'TV']
## we won't be looking at content-based filtering so we'll only keep the anime_id and name fields before
## performing a left join with the conventional ratings data
animes = animes.drop(animes.columns[2:], axis=1)

In [5]:
## filter the ratings dataframe to only contain TV animes
mask = ratings['anime_id'].isin(animes['anime_id'])
ratings = ratings[mask]

In [6]:
## left join on anime_id's then just use the name instead of ambigious id for interpretation
ratings = ratings.merge(animes, on='anime_id', how='left')
ratings= ratings[['user_id', 'name', 'rating']]
ratings.reset_index(drop=True, inplace=True)

In [7]:
ratings['rating'].value_counts()

 8     1163822
-1      919302
 7      917882
 9      915266
 10     689023
 6      401054
 5      169651
 4       62627
 3       24060
 2       12708
 1        8201
Name: rating, dtype: int64

In [8]:
## -1 values in the rating column indicate that the user watched the anime but did not rate it so 
## we'll replace these values with NaN
ratings['rating'].replace(-1, np.nan, inplace=True)

# Bias Subtracted Item-Item Based Collaborative Filtering
#### We will start with a simple recommender system using item-item collaborative filtering.
#### This approach is the easiest to implement because no training or optimization is required, however there are severe limitations.
#### These limitations include scalability because its computationally expensive to calculate the similarities between items, there is popularity bias towards popular items and there is a cold-start problem which means the recommender fails to recommend new or less-known items since they have little interactions.

In [9]:
## remedy to data sparsity, although this would hinder the performance of the recommender in general
## only keep users that have rated at least 20 animes 
ratings = ratings.groupby('user_id').filter(lambda x: x['rating'].count() >= 20)
## keep animes that have been rated at least 10 times
ratings = ratings.groupby('name').filter(lambda x: len(x) > 10)

In [10]:
## combine dataframes to a pivot table, essentially a matrix (R) where rows (i) represent users, 
## columns (j) represent the animes such that the entries r_ij is the rating user i assigned to anime j
R = ratings.pivot_table(index='user_id', columns='name', values ='rating')


## Ratings Matrix

In [11]:
R

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,Zombie-Loan,"Zone of the Enders: Dolores, I",ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73507,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,9.0
73510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
## for the sake of computation, take a random sample from the Ratings Matrix of 10000 users ratings
R = R.sample(n=10000, random_state=42)

In [13]:
## user means
user_ratings_mean = np.array(R.mean(axis=1))

In [14]:
## standardize to centre the mean about zero
## the intuition is that users may tend to always give high or low ratings to all animes
## We want the relative difference in the ratings that these users give
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [15]:
## fill NaN values with 0
R_demeaned = R.fillna(0).as_matrix()

#### We will use adjusted cosine similarity to calculate item similarity of items where U is the set of all users
$$\mathcal{sim(i, j)} = \cos(\theta) = \frac{\sum_{u \in U}(R_{u,i} - \bar{R_i})(R_{u,j} - \bar{R_j})}{\sqrt{\sum_{u \in U}(R_{u,i} - \bar{R_i})^2}{\sqrt{\sum_{u \in U}(R_{u,j} - \bar{R_j})^2}}}$$

In [16]:
## function for (item) cosine similarity
def my_cosine_similarity(ratings, epsilon=1e-9):
    sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [17]:
## calculate the item cosine similarity between (animes i and j for all animes)
item_similarity = my_cosine_similarity(R_demeaned)

In [18]:
## item cosine similarity matrix as a dataframe
item_similarity_df = pd.DataFrame(item_similarity, index=R.columns, columns=R.columns)

In [19]:
## function to find top 5 similar anime shows to a given anime
def get_similar_animes(anime_name):
    similar_animes = item_similarity_df.sort_values(by=anime_name, ascending=False).index[1:6]
    similarity_score = item_similarity_df.sort_values(by=anime_name, ascending=False).loc[:, anime_name].tolist()[1:6]
    return similar_animes, similarity_score

In [20]:
recommended_animes, scores = get_similar_animes('Steins;Gate')

In [21]:
print('5 similar animes to Steins;Gate\n')
for i, (anime, score) in enumerate(zip(recommended_animes, scores)):
      print('({}) {} ... with similarity of {}\n'.format(i+1, anime, score))

5 similar animes to Steins;Gate

(1) Shingeki no Kyojin ... with similarity of 0.5840221275212034

(2) Angel Beats! ... with similarity of 0.5829477861845317

(3) Fullmetal Alchemist: Brotherhood ... with similarity of 0.5798342840618599

(4) Fate/Zero ... with similarity of 0.5707470901295555

(5) Code Geass: Hangyaku no Lelouch ... with similarity of 0.570125701352429



# Collaborative Filtering using Matrix Factorization
#### Matrix Factorization is an embedding model. An embedding is a [relatively] low-dimensional space into which you can translate high-dimensional vectors. In recommender systems, the ratings matrix is approximated by the product of user and item embedding matrices. The model must learn the embeddings so that
$$\mathbf{R}\backsimeq\mathbf{P}\mathbf{Q}$$
#### where
$$\mathbf{R}\in\mathbb{R}^{m \times n},\, \mathbf{P}\in\mathbb{R}^{m \times k},\, \mathbf{Q}\in\mathbb{R}^{k \times n} $$

#### Matrix factorization models map both users and items to a joint latent factor space of dimensionality k, a hyperparameter that selects the number of latent factors to use. Latent factors are hidden features and can be thought of as a computerized alternative to tastes and preferences. For instance, a latent factor here may refer to the genre that the anime belongs to. Essentially we want to find a low-dimensional representation of users and animes such that people that like an anime are close to eachother.

#### As we've seen before, the NaN values in the Ratings matrix are important (ideally these are the values we want to predict). Matrix Factorization techniques such as Singular Value Decomposition are undefined for missing values and although we could impute the NaN values using a simple heuristic, it wouldn't make much sense and these models wouldn't outperform simple collaborative filtering techniques like the one we implemented above by any means.

#### To find P and Q we will frame this problem as an optimization problem which was used in the Netlfix Challenge. The loss function is defined below and is solved using Alternating Least Squares or Stochastic Gradient Descent in practice:
$$\min_{P, Q} \sum_{(i,j) \in R} (r_{ij} - q_i^\mathrm{T}{p_j})^2 + \lambda(\sum_{i}\left\lVert q_i\right\rVert^2 + \sum_{j}\left\lVert p_j\right\rVert^2)$$
#### i.e. minimize error on known ratings with regularization term to prevent overfitting.

In [22]:
## the surprise library is used for recommender systems and implements the SVD variant discussed above
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import cross_validate, train_test_split

## drop the NaNs from ratings
ratings = ratings.dropna()
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(ratings, reader)

In [23]:
## train test split
train, test = train_test_split(data, test_size=.25)

In [24]:
## probabilistic matrix factorization, inspired by singular value decomposition using 20 latent factors
matrix_factorization_algo = SVD(n_factors=20)

In [25]:
# train the algorithm and predict ratings for test
matrix_factorization_algo.fit(train)
predictions = matrix_factorization_algo.test(test)
accuracy.rmse(predictions)

RMSE: 1.1102


1.110170534524507

In [26]:
from collections import defaultdict

## function to get top 3 estimated ratings for a given user:
def top_three_pred(user_id):
    ## setup a dictionary with keys as user id's and values as (anime, predicted rating)
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        
    ## sort the values by top rated animes and keep top 3
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:3]
    return top_n[user_id]

In [27]:
for anime, rating in top_three_pred(31415):
    print(anime, rating)

Boku dake ga Inai Machi 8.6726972184522
Kiseijuu: Sei no Kakuritsu 8.506869273690286
Shingeki no Kyojin 8.504621550267139


#### Next steps would be to build a web app so that people could get new anime recommendations :)